## Module 1.3

In [10]:
import minsearch

In [11]:
import json

In [12]:
with open('documents.json',  'rt') as f_in:
    docs_raw = json.load(f_in)

In [13]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [14]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields=['course']
)

In [16]:
q = 'the course has already started, can I still enroll?'

In [17]:
index.fit(documents)

In [18]:
# default boost of 1.0
boost = {
    'question': 3.0,
    'section': 0.5
}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'course': 'data-engineering-zoomcamp'}
    
)

In [19]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Module 1.4

In [20]:
from google import genai
import os

GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

client = genai.Client(api_key=GEMINI_API_KEY)

In [21]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    # config=types.GenerateContentConfig(
        # system_instruction="You are a cat. Your name is Neko."),
    contents=q
)

In [22]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text="Whether or not you can still enroll in a course that has already started depends on several factors:\n\n*   **The course's policy:** Some courses have strict deadlines for enrollment, while others are more flexible. Look for information on late enrollment or contact the course provider (e.g., the school, university, or online platform).\n*   **The type of course:** Self-paced online courses are often more flexible, allowing enrollment at any time. Traditional in-person courses may have stricter enrollment deadlines.\n*   **The platform or institution:** Some platforms or institutions have specific policies about late enrollment, such as requiring permission from the instructor or paying a late fee.\n*   **The course's conte

In [23]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""

In [24]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

In [25]:
prompt = prompt_template.format(question=q, context=context).strip()

In [26]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    # config=types.GenerateContentConfig(
        # system_instruction="You are a cat. Your name is Neko."),
    contents=prompt
)

In [27]:
response.text

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.\n"

## Module 1.5

In [28]:
def search(query):
    # default boost of 1.0
    boost = {
        'question': 3.0,
        'section': 0.5
    }
    
    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=10,
        filter_dict={'course': 'data-engineering-zoomcamp'}
        
    )

    return results

In [29]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT does not contain the answer, output NONE
        
        QUESTION: {question}
        
        CONTEXT:
        {context}
        """
    
    context = ""

    for doc in results:
        context += f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [30]:
def llm(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        # config=types.GenerateContentConfig(
            # system_instruction="You are a cat. Your name is Neko."),
        contents=prompt
    )
    return response.text

In [31]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer


In [32]:
rag('Can I follow the course on Windows?')

'NONE\n'

## Module 1.6

In [33]:
from elasticsearch import Elasticsearch

es_url = 'http://127.0.0.1:9200/'

es_client = Elasticsearch(es_url)

In [34]:
es_client.info()

ObjectApiResponse({'name': '9e1cbcba1439', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'K-W8BupkQhWdW5_yhQQu2w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_21351/310768491.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [37]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [38]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)



100%|██████████████████████████████████████████████████| 948/948 [00:03<00:00, 241.36it/s]


In [42]:
query = "I just discovered the course. Can I still join?"

In [50]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        # field question is 3 times more important than the other ones
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [45]:
response = es_client.search(index=index_name, body=search_query)

/tmp/ipykernel_21351/449040615.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [51]:
elastic_search

<function __main__.elastic_search(query)>

In [52]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [53]:
rag(query)

/tmp/ipykernel_21351/3491281928.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.\n"